<a href="https://colab.research.google.com/github/Padawanh/GEE_analysis/blob/main/GiF_Analise_Colheita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap


In [ ]:
import ee
import geemap
import os
import datetime
Map = geemap.Map()

In [ ]:
#geemap.update_package()

In [48]:
#Coordenadas Talhão MA02WFCO3A
geoP = ee.Geometry.Point([-49.30104 , -25.05251])
start , end ,porcentagemNuvem  = '2021-04-01','2021-08-31' , 10

#Processamento

In [49]:
Map = geemap.Map()

pointColheita = ee.Feature(geoP)
roi = pointColheita.buffer(500)
geometry = geoP
Map.centerObject(geometry,20)
s2 = ee.ImageCollection("COPERNICUS/S2")


# Write a function for Cloud masking
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date(start , end )) \
  .filter(ee.Filter.intersects('.geo', geometry)) \
  .map(maskS2clouds).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',porcentagemNuvem))


imgCol = filtered.map(lambda img: img.clip(roi))

rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

'''
Map.addLayer(imgCol, rgbVis, 'RGB Composite')
Map.addLayer(geometry,name='Amostras')
Map
'''

"\nMap.addLayer(imgCol, rgbVis, 'RGB Composite')\nMap.addLayer(geometry,name='Amostras')\nMap\n"

In [50]:
print(imgCol.size().getInfo())

6


In [51]:

time = [str(datetime.datetime.fromtimestamp(x/1000).date()) for x in imgCol.aggregate_array('system:time_start').getInfo()]

In [52]:

work_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
out_gif = os.path.join(work_dir, "landsat_ts.gif")
geemap.download_ee_video(imgCol, rgbVis, out_gif)
texted_gif = os.path.join(work_dir, "landsat_ts_text.gif")
geemap.add_text_to_gif(out_gif, texted_gif, xy=('3%', '5%'), text_sequence=time, font_size=15, font_color='#ffffff', add_progress_bar=False,duration = 1250)

Generating URL...
Please wait ...
The GIF image has been saved to: /root/Downloads/landsat_ts.gif


#Resultado

In [53]:
geemap.show_image(texted_gif)

Output()